In [11]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from matplotlib import pyplot as plt
from IPython.display import clear_output
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.models import load_model
%matplotlib inline
# Load the TensorBoard notebook extension.
%load_ext tensorboard
#from keras_radam import RAdam
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn as sk
import time
import sys
import collections
# Keras turner imports
import kerastuner
from kerastuner.tuners import RandomSearch, BayesianOptimization 
from kerastuner.engine.hyperparameters import HyperParameters

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [12]:
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.1.0
Keras Version: 2.2.4-tf

Python 3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]
Pandas 1.2.1
Scikit-Learn 0.24.1
GPU is available


In [ ]:
run_directory = "Z:\\FNS_GA_v5_100_gen_CNN_hyperparam_work\\models\\\integral_keff\\models"
data_directory = "Z:\\FNS_GA_v5_100_gen_CNN_hyperparam_work\\models\\\integral_keff\\data"
train_data_string = ["X_int_exp_training_data.csv.npy",
                     "Y_int_exp_training_data.csv.npy"]
validation_data_string = ["X_int_exp_validation_data.csv.npy",
                          "Y_int_exp_validation_data.csv.npy"]

In [13]:
#data_ = np.loadtxt("FNS_GA_Data_Runs_11_12_Paper_2.csv", delimiter=",")
#X = data_[:,0:150]
Y_train = np.load("FNS_int_keff_Y_3_opt_training_thru_gen_46.npy")
X_train = np.load("FNS_int_keff_X_3_opt_training_thru_gen_46.npy")

#data_test = np.loadtxt("FNS_GA_Run_13_Data_Test.csv", delimiter=",")
Y_test = np.load("FNS_int_keff_Y_3_opt_validation_gen_47_to_49.npy")
X_test = np.load("FNS_int_keff_X_3_opt_validation_gen_47_to_49.npy")
#X_test = np.load("test_2x2_fns_run_test_dat.npy")

In [14]:
from kerastuner import HyperModel
class HyperModel_555_v2(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape
    def build(self, hp):
        conv_size = hp.Int("first_conv_size", min_value=16, max_value=64, step=1)
        kernel_size = hp.Int("kernel_size", min_value=4, max_value=10, step=1)
        dense_layer_size = hp.Int("dense_layer_size", min_value=32, max_value=128, step=32)
        max_pool_val = hp.Int("max_pool_val", min_value=2, max_value=10, step=1)

        model = Sequential()
        model.add(Conv3D(conv_size, kernel_size=(kernel_size, 2, 2),
                         input_shape=self.input_shape, padding='same', data_format="channels_last", strides = (1, 1, 1)))
        model.add(LeakyReLU(alpha=0.3))
        model.add(BatchNormalization())
        ##model.add(Conv3D(32, (3, 3, 5), activation=LeakyReLU(alpha=0.3), padding='same'))

        for i in range(hp.Int("n_layers", 7, 12)):
            model.add(Conv3D(hp.Int(f"conv_{i}_size",
                                    min_value=16,
                                    max_value=64,
                                    step=8),
                             (kernel_size, 2, 2),
                             padding='same'))
            model.add(LeakyReLU(alpha=0.3))
            model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(max_pool_val, 2, 2)))
        model.add(Flatten())
        model.add(BatchNormalization())
        model.add(Dense(dense_layer_size))
        model.add(LeakyReLU(alpha=0.3))
        model.add(BatchNormalization())
        model.add(Dense(dense_layer_size))
        model.add(LeakyReLU(alpha=0.3))
        model.add(Dropout(hp.Float('dropout_value', min_value = 0.0, max_value = 0.4, step=0.1)))
        
        model.add(Dense(1, activation='linear'))

        ### Building the optimizer object
        #sgd = optimizers.SGD(lr = 0.1) # 1.0 0.0001 0.01
        opt = optimizers.Adam(lr = hp.Float(
                    'learning_rate',
                    min_value=0.0001,
                    max_value=0.04,
                    default=0.0001,
                    step=1e-4)) 
           
        ### Adding the loss function and optimizer to the model
        #mcp_save = ModelCheckpoint('CNN_3d_2x2_test_2.hdf5', save_best_only=True, monitor='val_loss', mode='min')
        #opt = RAdam(total_steps=100, warmup_proportion=0.1, min_lr=1e-5)
        model.compile(loss='mse', optimizer=opt, metrics=['mae'])
        ### Running the model


        return model

In [23]:
input_shape = (60, 2, 2, 4)
hypermodel = HyperModel_555_v2(input_shape)

In [24]:
LOG_DIR = "bayes_int_exp_1_"
max_trials_val = 100
# int(max_trials_val/10)
tuner_bo = BayesianOptimization(
            hypermodel,
            objective='val_mae',
            max_trials=max_trials_val,
            seed=1,
            executions_per_trial=2,
            num_initial_points = int(max_trials_val/10),
            directory = LOG_DIR)
#tuner_bo.search(x=X_train, y=Y_train,
#             epochs=300,
#             batch_size=2000,
#             validation_data=(X_test, Y_test))

INFO:tensorflow:Reloading Oracle from existing project bayes_test_keff_gen_46_2_\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from bayes_test_keff_gen_46_2_\untitled_project\tuner0.json


In [17]:
#tuner_bo.results_summary()

In [18]:
#best_model = tuner_bo.get_best_models(num_models=10)
#best_model[0].save('best_model_bayes_test_keff_gen_46_2')

In [25]:
def train_top_models(models_list,
                     model_string_template = "test",
                     csv_logger_bool = True,
                     model_checkpoint_boo = True,
                     save_original_model = True,
                     train_models = True,
                     train_model_options = {}):
    
    for model_count, current_model in enumerate(models_list):
        model_string = model_string_template + "_model_" + str(model_count)
        
        callback_list = []
        if csv_logger_bool:
            csv_logger = CSVLogger(model_string + '.log')
            callback_list.append(csv_logger)
            
        if model_checkpoint_boo:
            checkpoint_filepath = model_string + "_chkpnt_.{epoch:02d}-{val_mae:.6f}.hdf5"
            model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                            filepath=checkpoint_filepath,
                                            monitor='val_mae',
                                            mode='min',
                                            save_best_only=True,
                                            verbose = 1)
            callback_list.append(model_checkpoint_callback)
            
        if save_original_model:
            current_model.save(model_string + '_original')
            
        if train_models:
            current_model.fit(train_model_options['x_train'],
                 train_model_options['y_train'],
                 validation_data=train_model_options['validation_data'],
                 epochs=train_model_options['epochs'],
                 batch_size=train_model_options['batch_size'],
                 verbose=train_model_options['verbose'],
                 callbacks=callback_list)

In [26]:
best_models = tuner_bo.get_best_models(num_models=10)
train_model_opt = collections.OrderedDict()
train_model_opt['x_train'] = X_train
train_model_opt['y_train'] = Y_train
train_model_opt['validation_data'] = (X_test, Y_test)
train_model_opt['epochs'] = 300
train_model_opt['batch_size'] = int(len(X_train)/4)
train_model_opt['verbose'] = 1


train_top_models(best_models, train_model_options = train_model_opt,model_string_template = "int_keff")

INFO:tensorflow:Assets written to: int_keff_model_0_original\assets
Train on 3339 samples, validate on 194 samples
Epoch 1/300
3336/3339 [============================>.] - ETA: 8s - loss: 0.0323 - mae: 0.141 - ETA: 2s - loss: 0.0314 - mae: 0.140 - ETA: 0s - loss: 0.0310 - mae: 0.140 - ETA: 0s - loss: 0.0312 - mae: 0.1402
Epoch 00001: val_mae improved from inf to 0.19002, saving model to int_keff_model_0_chkpnt_.01-0.190015.hdf5
3339/3339 [==============================] - 3s 1ms/sample - loss: 0.0312 - mae: 0.1402 - val_loss: 0.0373 - val_mae: 0.1900
Epoch 2/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0329 - mae: 0.147 - ETA: 0s - loss: 0.0330 - mae: 0.145 - ETA: 0s - loss: 0.0341 - mae: 0.147 - ETA: 0s - loss: 0.0355 - mae: 0.1498
Epoch 00002: val_mae improved from 0.19002 to 0.15291, saving model to int_keff_model_0_chkpnt_.02-0.152907.hdf5
3339/3339 [==============================] - 0s 127us/sample - loss: 0.0355 - mae: 0.1500 - val_loss: 0.0245 - val_mae: 0.

3336/3339 [============================>.] - ETA: 0s - loss: 0.0175 - mae: 0.104 - ETA: 0s - loss: 0.0174 - mae: 0.104 - ETA: 0s - loss: 0.0176 - mae: 0.105 - ETA: 0s - loss: 0.0182 - mae: 0.1071
Epoch 00042: val_mae did not improve from 0.04809
3339/3339 [==============================] - 0s 106us/sample - loss: 0.0182 - mae: 0.1071 - val_loss: 0.0050 - val_mae: 0.0572
Epoch 43/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0188 - mae: 0.107 - ETA: 0s - loss: 0.0186 - mae: 0.107 - ETA: 0s - loss: 0.0181 - mae: 0.106 - ETA: 0s - loss: 0.0180 - mae: 0.1061
Epoch 00043: val_mae did not improve from 0.04809
3339/3339 [==============================] - 0s 106us/sample - loss: 0.0180 - mae: 0.1061 - val_loss: 0.0045 - val_mae: 0.0533
Epoch 44/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0158 - mae: 0.100 - ETA: 0s - loss: 0.0167 - mae: 0.102 - ETA: 0s - loss: 0.0170 - mae: 0.103 - ETA: 0s - loss: 0.0168 - mae: 0.1026
Epoch 00044: val_mae improved fr

Epoch 63/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0128 - mae: 0.090 - ETA: 0s - loss: 0.0125 - mae: 0.089 - ETA: 0s - loss: 0.0123 - mae: 0.088 - ETA: 0s - loss: 0.0122 - mae: 0.0877
Epoch 00063: val_mae did not improve from 0.04489
3339/3339 [==============================] - 0s 108us/sample - loss: 0.0122 - mae: 0.0876 - val_loss: 0.0134 - val_mae: 0.1049
Epoch 64/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0124 - mae: 0.088 - ETA: 0s - loss: 0.0123 - mae: 0.088 - ETA: 0s - loss: 0.0122 - mae: 0.088 - ETA: 0s - loss: 0.0124 - mae: 0.0889
Epoch 00064: val_mae did not improve from 0.04489
3339/3339 [==============================] - 0s 106us/sample - loss: 0.0124 - mae: 0.0889 - val_loss: 0.0125 - val_mae: 0.1011
Epoch 65/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0109 - mae: 0.082 - ETA: 0s - loss: 0.0117 - mae: 0.085 - ETA: 0s - loss: 0.0116 - mae: 0.085 - ETA: 0s - loss: 0.0117 - mae: 0.0857
Epoch 00065: val_ma

Epoch 00104: val_mae improved from 0.03489 to 0.03335, saving model to int_keff_model_0_chkpnt_.104-0.033345.hdf5
3339/3339 [==============================] - 0s 126us/sample - loss: 0.0075 - mae: 0.0680 - val_loss: 0.0019 - val_mae: 0.0333
Epoch 105/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0075 - mae: 0.069 - ETA: 0s - loss: 0.0081 - mae: 0.071 - ETA: 0s - loss: 0.0083 - mae: 0.072 - ETA: 0s - loss: 0.0084 - mae: 0.0722
Epoch 00105: val_mae did not improve from 0.03335
3339/3339 [==============================] - 0s 106us/sample - loss: 0.0084 - mae: 0.0722 - val_loss: 0.0036 - val_mae: 0.0478
Epoch 106/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0090 - mae: 0.074 - ETA: 0s - loss: 0.0089 - mae: 0.074 - ETA: 0s - loss: 0.0087 - mae: 0.073 - ETA: 0s - loss: 0.0088 - mae: 0.0738
Epoch 00106: val_mae did not improve from 0.03335
3339/3339 [==============================] - 0s 105us/sample - loss: 0.0088 - mae: 0.0738 - val_loss: 0.0042 - v

Epoch 00146: val_mae did not improve from 0.03140
3339/3339 [==============================] - 0s 109us/sample - loss: 0.0053 - mae: 0.0575 - val_loss: 0.0020 - val_mae: 0.0375
Epoch 147/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0056 - mae: 0.059 - ETA: 0s - loss: 0.0056 - mae: 0.060 - ETA: 0s - loss: 0.0057 - mae: 0.060 - ETA: 0s - loss: 0.0058 - mae: 0.0608
Epoch 00147: val_mae did not improve from 0.03140
3339/3339 [==============================] - 0s 108us/sample - loss: 0.0058 - mae: 0.0608 - val_loss: 0.0017 - val_mae: 0.0339
Epoch 148/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0048 - mae: 0.055 - ETA: 0s - loss: 0.0052 - mae: 0.056 - ETA: 0s - loss: 0.0053 - mae: 0.057 - ETA: 0s - loss: 0.0053 - mae: 0.0574
Epoch 00148: val_mae did not improve from 0.03140
3339/3339 [==============================] - 0s 105us/sample - loss: 0.0053 - mae: 0.0574 - val_loss: 0.0017 - val_mae: 0.0327
Epoch 149/300
3336/3339 [========================

...

Epoch 119/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0013 - mae: 0.028 - ETA: 0s - loss: 0.0013 - mae: 0.028 - ETA: 0s - loss: 0.0012 - mae: 0.028 - ETA: 0s - loss: 0.0012 - mae: 0.0279
Epoch 00119: val_mae did not improve from 0.02180
3339/3339 [==============================] - 0s 95us/sample - loss: 0.0012 - mae: 0.0279 - val_loss: 0.0015 - val_mae: 0.0311
Epoch 120/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0012 - mae: 0.026 - ETA: 0s - loss: 0.0012 - mae: 0.026 - ETA: 0s - loss: 0.0012 - mae: 0.026 - ETA: 0s - loss: 0.0012 - mae: 0.0269
Epoch 00120: val_mae did not improve from 0.02180
3339/3339 [==============================] - 0s 94us/sample - loss: 0.0012 - mae: 0.0269 - val_loss: 0.0012 - val_mae: 0.0283
Epoch 121/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0011 - mae: 0.026 - ETA: 0s - loss: 0.0012 - mae: 0.027 - ETA: 0s - loss: 0.0012 - mae: 0.027 - ETA: 0s - loss: 0.0012 - mae: 0.0273
Epoch 00121: val_m

3336/3339 [============================>.] - ETA: 0s - loss: 0.0015 - mae: 0.029 - ETA: 0s - loss: 0.0014 - mae: 0.029 - ETA: 0s - loss: 0.0014 - mae: 0.028 - ETA: 0s - loss: 0.0013 - mae: 0.0283
Epoch 00161: val_mae did not improve from 0.02180
3339/3339 [==============================] - 0s 95us/sample - loss: 0.0013 - mae: 0.0283 - val_loss: 0.0020 - val_mae: 0.0367
Epoch 162/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0012 - mae: 0.027 - ETA: 0s - loss: 0.0014 - mae: 0.028 - ETA: 0s - loss: 0.0015 - mae: 0.029 - ETA: 0s - loss: 0.0015 - mae: 0.0295
Epoch 00162: val_mae did not improve from 0.02180
3339/3339 [==============================] - 0s 95us/sample - loss: 0.0015 - mae: 0.0294 - val_loss: 9.6963e-04 - val_mae: 0.0236
Epoch 163/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0014 - mae: 0.028 - ETA: 0s - loss: 0.0014 - mae: 0.028 - ETA: 0s - loss: 0.0013 - mae: 0.028 - ETA: 0s - loss: 0.0013 - mae: 0.0282
Epoch 00163: val_mae did not

3336/3339 [============================>.] - ETA: 0s - loss: 0.0011 - mae: 0.025 - ETA: 0s - loss: 0.0011 - mae: 0.026 - ETA: 0s - loss: 0.0011 - mae: 0.026 - ETA: 0s - loss: 0.0011 - mae: 0.0260
Epoch 00202: val_mae did not improve from 0.01987
3339/3339 [==============================] - 0s 95us/sample - loss: 0.0011 - mae: 0.0260 - val_loss: 0.0020 - val_mae: 0.0362
Epoch 203/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0012 - mae: 0.026 - ETA: 0s - loss: 0.0013 - mae: 0.027 - ETA: 0s - loss: 0.0014 - mae: 0.028 - ETA: 0s - loss: 0.0014 - mae: 0.0290
Epoch 00203: val_mae did not improve from 0.01987
3339/3339 [==============================] - 0s 94us/sample - loss: 0.0014 - mae: 0.0290 - val_loss: 9.7933e-04 - val_mae: 0.0238
Epoch 204/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0012 - mae: 0.028 - ETA: 0s - loss: 0.0012 - mae: 0.028 - ETA: 0s - loss: 0.0011 - mae: 0.027 - ETA: 0s - loss: 0.0011 - mae: 0.0269
Epoch 00204: val_mae did not

3336/3339 [============================>.] - ETA: 0s - loss: 9.3594e-04 - mae: 0.024 - ETA: 0s - loss: 9.5050e-04 - mae: 0.024 - ETA: 0s - loss: 9.4267e-04 - mae: 0.023 - ETA: 0s - loss: 9.5665e-04 - mae: 0.0238
Epoch 00243: val_mae did not improve from 0.01987
3339/3339 [==============================] - 0s 95us/sample - loss: 9.5780e-04 - mae: 0.0238 - val_loss: 0.0017 - val_mae: 0.0359
Epoch 244/300
3336/3339 [============================>.] - ETA: 0s - loss: 8.3359e-04 - mae: 0.022 - ETA: 0s - loss: 7.9655e-04 - mae: 0.022 - ETA: 0s - loss: 7.8932e-04 - mae: 0.022 - ETA: 0s - loss: 7.9975e-04 - mae: 0.0223
Epoch 00244: val_mae did not improve from 0.01987
3339/3339 [==============================] - 0s 95us/sample - loss: 7.9954e-04 - mae: 0.0223 - val_loss: 8.3844e-04 - val_mae: 0.0240
Epoch 245/300
3336/3339 [============================>.] - ETA: 0s - loss: 7.2640e-04 - mae: 0.021 - ETA: 0s - loss: 6.9091e-04 - mae: 0.020 - ETA: 0s - loss: 6.9426e-04 - mae: 0.020 - ETA: 0s - los

3339/3339 [==============================] - 0s 95us/sample - loss: 0.0011 - mae: 0.0268 - val_loss: 0.0057 - val_mae: 0.0707
Epoch 285/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0011 - mae: 0.026 - ETA: 0s - loss: 0.0011 - mae: 0.026 - ETA: 0s - loss: 0.0011 - mae: 0.026 - ETA: 0s - loss: 0.0011 - mae: 0.0258
Epoch 00285: val_mae did not improve from 0.01838
3339/3339 [==============================] - 0s 96us/sample - loss: 0.0011 - mae: 0.0258 - val_loss: 0.0093 - val_mae: 0.0933
Epoch 286/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0011 - mae: 0.026 - ETA: 0s - loss: 0.0010 - mae: 0.025 - ETA: 0s - loss: 9.7330e-04 - mae: 0.024 - ETA: 0s - loss: 9.7938e-04 - mae: 0.0243
Epoch 00286: val_mae did not improve from 0.01838
3339/3339 [==============================] - 0s 96us/sample - loss: 9.7977e-04 - mae: 0.0243 - val_loss: 0.0039 - val_mae: 0.0569
Epoch 287/300
3336/3339 [============================>.] - ETA: 0s - loss: 0.0010 - mae: 0

In [27]:
### Pick up the best from the keff study, train it on the new data.
### model_6_model_0_chkpnt_.312-0.008630
loaded_model = keras.models.load_model("int_keff_model_9_chkpnt_.246-0.018383.hdf5")
print(loaded_model.summary())
output_ = loaded_model.predict(X_test)
for real, pred in zip(Y_test, output_):
    print(real[0], pred[0])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 60, 2, 2, 16)      784       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 60, 2, 2, 16)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 60, 2, 2, 16)      64        
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 60, 2, 2, 24)      4632      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 60, 2, 2, 24)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 60, 2, 2, 24)      96        
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 60, 2, 2, 16)      4

In [28]:

train_model_opt = collections.OrderedDict()
train_model_opt['x_train'] = X_train
train_model_opt['y_train'] = Y_train
train_model_opt['validation_data'] = (X_test, Y_test)
train_model_opt['epochs'] = 3000
train_model_opt['batch_size'] = int(len(X_train)/4)
train_model_opt['verbose'] = 1


train_top_models([loaded_model], model_string_template = "best_keff_model_for_int_keff", train_model_options = train_model_opt)

INFO:tensorflow:Assets written to: best_keff_model_for_int_keff_model_0_original\assets
Train on 3339 samples, validate on 194 samples
Epoch 1/3000
3336/3339 [============================>.] - ETA: 3s - loss: 7.7828e-04 - mae: 0.022 - ETA: 1s - loss: 0.0075 - mae: 0.0646    - ETA: 0s - loss: 0.0070 - mae: 0.063 - ETA: 0s - loss: 0.0065 - mae: 0.0621
Epoch 00001: val_mae improved from inf to 0.08206, saving model to best_keff_model_for_int_keff_model_0_chkpnt_.01-0.082062.hdf5
3339/3339 [==============================] - 2s 540us/sample - loss: 0.0064 - mae: 0.0621 - val_loss: 0.0079 - val_mae: 0.0821
Epoch 2/3000
3336/3339 [============================>.] - ETA: 0s - loss: 0.0031 - mae: 0.041 - ETA: 0s - loss: 0.0030 - mae: 0.040 - ETA: 0s - loss: 0.0029 - mae: 0.040 - ETA: 0s - loss: 0.0028 - mae: 0.0400
Epoch 00002: val_mae did not improve from 0.08206
3339/3339 [==============================] - 0s 97us/sample - loss: 0.0028 - mae: 0.0400 - val_loss: 0.0536 - val_mae: 0.2181
Epoch 3

3336/3339 [============================>.] - ETA: 0s - loss: 0.0020 - mae: 0.036 - ETA: 0s - loss: 0.0026 - mae: 0.041 - ETA: 0s - loss: 0.0024 - mae: 0.040 - ETA: 0s - loss: 0.0021 - mae: 0.0376
Epoch 00042: val_mae did not improve from 0.08206
3339/3339 [==============================] - 0s 97us/sample - loss: 0.0021 - mae: 0.0376 - val_loss: 0.2282 - val_mae: 0.4511
Epoch 43/3000
3336/3339 [============================>.] - ETA: 0s - loss: 0.0028 - mae: 0.039 - ETA: 0s - loss: 0.0033 - mae: 0.043 - ETA: 0s - loss: 0.0032 - mae: 0.042 - ETA: 0s - loss: 0.0028 - mae: 0.0392
Epoch 00043: val_mae did not improve from 0.08206
3339/3339 [==============================] - 0s 96us/sample - loss: 0.0028 - mae: 0.0392 - val_loss: 0.7029 - val_mae: 0.7898
Epoch 44/3000
3336/3339 [============================>.] - ETA: 0s - loss: 0.0012 - mae: 0.026 - ETA: 0s - loss: 0.0012 - mae: 0.027 - ETA: 0s - loss: 0.0013 - mae: 0.027 - ETA: 0s - loss: 0.0012 - mae: 0.0267
Epoch 00044: val_mae did not imp

3336/3339 [============================>.] - ETA: 0s - loss: 5.9625e-04 - mae: 0.019 - ETA: 0s - loss: 6.4263e-04 - mae: 0.019 - ETA: 0s - loss: 6.6509e-04 - mae: 0.020 - ETA: 0s - loss: 6.7899e-04 - mae: 0.0203
Epoch 00083: val_mae did not improve from 0.03369
3339/3339 [==============================] - 0s 96us/sample - loss: 6.7847e-04 - mae: 0.0202 - val_loss: 0.0816 - val_mae: 0.2737
Epoch 84/3000
3336/3339 [============================>.] - ETA: 0s - loss: 7.2133e-04 - mae: 0.019 - ETA: 0s - loss: 6.8496e-04 - mae: 0.019 - ETA: 0s - loss: 6.9808e-04 - mae: 0.020 - ETA: 0s - loss: 6.6430e-04 - mae: 0.0197
Epoch 00084: val_mae did not improve from 0.03369
3339/3339 [==============================] - 0s 95us/sample - loss: 6.6393e-04 - mae: 0.0197 - val_loss: 0.0410 - val_mae: 0.1977
Epoch 85/3000
3336/3339 [============================>.] - ETA: 0s - loss: 6.2636e-04 - mae: 0.020 - ETA: 0s - loss: 7.4861e-04 - mae: 0.022 - ETA: 0s - loss: 7.9368e-04 - mae: 0.023 - ETA: 0s - loss: 7

3339/3339 [==============================] - 0s 94us/sample - loss: 0.0010 - mae: 0.0251 - val_loss: 0.0610 - val_mae: 0.2378
Epoch 124/3000
3336/3339 [============================>.] - ETA: 0s - loss: 8.7463e-04 - mae: 0.022 - ETA: 0s - loss: 8.6353e-04 - mae: 0.022 - ETA: 0s - loss: 8.4585e-04 - mae: 0.022 - ETA: 0s - loss: 8.1005e-04 - mae: 0.0218
Epoch 00124: val_mae did not improve from 0.01841
3339/3339 [==============================] - 0s 95us/sample - loss: 8.1097e-04 - mae: 0.0218 - val_loss: 0.0103 - val_mae: 0.0980
Epoch 125/3000
3336/3339 [============================>.] - ETA: 0s - loss: 0.0014 - mae: 0.027 - ETA: 0s - loss: 0.0019 - mae: 0.031 - ETA: 0s - loss: 0.0019 - mae: 0.030 - ETA: 0s - loss: 0.0017 - mae: 0.0290
Epoch 00125: val_mae did not improve from 0.01841
3339/3339 [==============================] - 0s 95us/sample - loss: 0.0017 - mae: 0.0290 - val_loss: 0.0723 - val_mae: 0.2636
Epoch 126/3000
3336/3339 [============================>.] - ETA: 0s - loss: 0.00

3339/3339 [==============================] - 0s 95us/sample - loss: 0.0020 - mae: 0.0354 - val_loss: 0.0104 - val_mae: 0.0980
Epoch 166/3000
3336/3339 [============================>.] - ETA: 0s - loss: 0.0051 - mae: 0.059 - ETA: 0s - loss: 0.0046 - mae: 0.054 - ETA: 0s - loss: 0.0034 - mae: 0.044 - ETA: 0s - loss: 0.0029 - mae: 0.0416
Epoch 00166: val_mae did not improve from 0.01841
3339/3339 [==============================] - 0s 95us/sample - loss: 0.0029 - mae: 0.0416 - val_loss: 0.0170 - val_mae: 0.1267
Epoch 167/3000
3336/3339 [============================>.] - ETA: 0s - loss: 0.0012 - mae: 0.027 - ETA: 0s - loss: 0.0012 - mae: 0.026 - ETA: 0s - loss: 0.0013 - mae: 0.027 - ETA: 0s - loss: 0.0015 - mae: 0.0297
Epoch 00167: val_mae did not improve from 0.01841
3339/3339 [==============================] - 0s 95us/sample - loss: 0.0015 - mae: 0.0297 - val_loss: 0.0371 - val_mae: 0.1908
Epoch 168/3000
3336/3339 [============================>.] - ETA: 0s - loss: 0.0014 - mae: 0.027 - ET

...

3336/3339 [============================>.] - ETA: 0s - loss: 0.0794 - mae: 0.228 - ETA: 0s - loss: 0.0524 - mae: 0.178 - ETA: 0s - loss: 0.0429 - mae: 0.159 - ETA: 0s - loss: 0.0362 - mae: 0.1446
Epoch 02815: val_mae did not improve from 0.01067
3339/3339 [==============================] - 0s 95us/sample - loss: 0.0362 - mae: 0.1446 - val_loss: 0.1145 - val_mae: 0.2489
Epoch 2816/3000
3336/3339 [============================>.] - ETA: 0s - loss: 0.0391 - mae: 0.158 - ETA: 0s - loss: 0.0240 - mae: 0.115 - ETA: 0s - loss: 0.0201 - mae: 0.106 - ETA: 0s - loss: 0.0167 - mae: 0.0961
Epoch 02816: val_mae did not improve from 0.01067
3339/3339 [==============================] - 0s 95us/sample - loss: 0.0167 - mae: 0.0961 - val_loss: 0.1731 - val_mae: 0.3206
Epoch 2817/3000
3336/3339 [============================>.] - ETA: 0s - loss: 0.0084 - mae: 0.073 - ETA: 0s - loss: 0.0104 - mae: 0.081 - ETA: 0s - loss: 0.0097 - mae: 0.078 - ETA: 0s - loss: 0.0087 - mae: 0.0736
Epoch 02817: val_mae did not

3336/3339 [============================>.] - ETA: 0s - loss: 5.1429e-04 - mae: 0.017 - ETA: 0s - loss: 5.7411e-04 - mae: 0.018 - ETA: 0s - loss: 5.9578e-04 - mae: 0.018 - ETA: 0s - loss: 5.8299e-04 - mae: 0.0187
Epoch 02856: val_mae did not improve from 0.01067
3339/3339 [==============================] - 0s 95us/sample - loss: 5.8483e-04 - mae: 0.0187 - val_loss: 0.0064 - val_mae: 0.0761
Epoch 2857/3000
3336/3339 [============================>.] - ETA: 0s - loss: 8.0532e-04 - mae: 0.022 - ETA: 0s - loss: 9.8953e-04 - mae: 0.025 - ETA: 0s - loss: 0.0011 - mae: 0.0272    - ETA: 0s - loss: 0.0011 - mae: 0.0277
Epoch 02857: val_mae did not improve from 0.01067
3339/3339 [==============================] - 0s 95us/sample - loss: 0.0011 - mae: 0.0277 - val_loss: 0.0028 - val_mae: 0.0492
Epoch 2858/3000
3336/3339 [============================>.] - ETA: 0s - loss: 4.7037e-04 - mae: 0.016 - ETA: 0s - loss: 5.5707e-04 - mae: 0.018 - ETA: 0s - loss: 6.1776e-04 - mae: 0.019 - ETA: 0s - loss: 7.170

3336/3339 [============================>.] - ETA: 0s - loss: 4.5095e-04 - mae: 0.016 - ETA: 0s - loss: 5.4380e-04 - mae: 0.018 - ETA: 0s - loss: 5.6037e-04 - mae: 0.019 - ETA: 0s - loss: 5.4018e-04 - mae: 0.0185
Epoch 02896: val_mae did not improve from 0.01067
3339/3339 [==============================] - 0s 95us/sample - loss: 5.4307e-04 - mae: 0.0185 - val_loss: 0.0013 - val_mae: 0.0329
Epoch 2897/3000
3336/3339 [============================>.] - ETA: 0s - loss: 5.0269e-04 - mae: 0.017 - ETA: 0s - loss: 6.0766e-04 - mae: 0.019 - ETA: 0s - loss: 6.8928e-04 - mae: 0.020 - ETA: 0s - loss: 7.0080e-04 - mae: 0.0205
Epoch 02897: val_mae did not improve from 0.01067
3339/3339 [==============================] - 0s 95us/sample - loss: 7.0124e-04 - mae: 0.0205 - val_loss: 6.0561e-04 - val_mae: 0.0203
Epoch 2898/3000
3336/3339 [============================>.] - ETA: 0s - loss: 3.9386e-04 - mae: 0.015 - ETA: 0s - loss: 5.5852e-04 - mae: 0.018 - ETA: 0s - loss: 7.3684e-04 - mae: 0.021 - ETA: 0s -

3336/3339 [============================>.] - ETA: 0s - loss: 7.0161e-04 - mae: 0.018 - ETA: 0s - loss: 7.0097e-04 - mae: 0.019 - ETA: 0s - loss: 7.8394e-04 - mae: 0.020 - ETA: 0s - loss: 7.7409e-04 - mae: 0.0212
Epoch 02936: val_mae did not improve from 0.01067
3339/3339 [==============================] - 0s 98us/sample - loss: 8.0674e-04 - mae: 0.0213 - val_loss: 0.0013 - val_mae: 0.0333
Epoch 2937/3000
3336/3339 [============================>.] - ETA: 0s - loss: 0.0038 - mae: 0.055 - ETA: 0s - loss: 0.0056 - mae: 0.065 - ETA: 0s - loss: 0.0053 - mae: 0.064 - ETA: 0s - loss: 0.0043 - mae: 0.0566
Epoch 02937: val_mae did not improve from 0.01067
3339/3339 [==============================] - 0s 96us/sample - loss: 0.0043 - mae: 0.0566 - val_loss: 0.0010 - val_mae: 0.0245
Epoch 2938/3000
3336/3339 [============================>.] - ETA: 0s - loss: 0.0033 - mae: 0.042 - ETA: 0s - loss: 0.0039 - mae: 0.045 - ETA: 0s - loss: 0.0040 - mae: 0.046 - ETA: 0s - loss: 0.0037 - mae: 0.0446
Epoch 02

Epoch 2976/3000
3336/3339 [============================>.] - ETA: 0s - loss: 4.4078e-04 - mae: 0.015 - ETA: 0s - loss: 4.8513e-04 - mae: 0.016 - ETA: 0s - loss: 5.2263e-04 - mae: 0.017 - ETA: 0s - loss: 5.1215e-04 - mae: 0.0174
Epoch 02976: val_mae did not improve from 0.01067
3339/3339 [==============================] - 0s 95us/sample - loss: 5.1195e-04 - mae: 0.0174 - val_loss: 5.6531e-04 - val_mae: 0.0203
Epoch 2977/3000
3336/3339 [============================>.] - ETA: 0s - loss: 3.1225e-04 - mae: 0.013 - ETA: 0s - loss: 3.2636e-04 - mae: 0.014 - ETA: 0s - loss: 3.6888e-04 - mae: 0.014 - ETA: 0s - loss: 3.9693e-04 - mae: 0.0155
Epoch 02977: val_mae did not improve from 0.01067
3339/3339 [==============================] - 0s 97us/sample - loss: 3.9780e-04 - mae: 0.0155 - val_loss: 8.8119e-04 - val_mae: 0.0274
Epoch 2978/3000
3336/3339 [============================>.] - ETA: 0s - loss: 4.2661e-04 - mae: 0.015 - ETA: 0s - loss: 5.1728e-04 - mae: 0.017 - ETA: 0s - loss: 6.4040e-04 - ma

In [29]:
loaded_model = keras.models.load_model("best_keff_model_for_int_keff_model_0_chkpnt_.2108-0.010675.hdf5")
print(loaded_model.summary())
output_ = loaded_model.predict(X_test)
for real, pred in zip(Y_test, output_):
    print(real[0], pred[0])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 60, 2, 2, 16)      784       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 60, 2, 2, 16)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 60, 2, 2, 16)      64        
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 60, 2, 2, 24)      4632      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 60, 2, 2, 24)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 60, 2, 2, 24)      96        
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 60, 2, 2, 16)      4

In [ ]:
loaded_model = keras.models.load_model("model_6_model_0_chkpnt_.338-0.007420.hdf5")
output_ =loaded_model.predict(X_train)
for real, pred in zip(Y_train, output_):
    print(real[0], pred[0])

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_mae', factor=0.5,
                              patience=150, min_lr=0.00001, verbose = 1)
csv_logger = CSVLogger('keff_long_training.log')

checkpoint_filepath = ''
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_mae',
    mode='min',
    save_best_only=True,
    verbose = 1)


In [ ]:
loaded_model.fit(X_train,
                 Y_train,
                 validation_data=(X_test, Y_test),
                 epochs=3000,
                 batch_size=1000,
                 verbose=1,
                 callbacks=[reduce_lr, csv_logger, model_checkpoint_callback])

In [ ]:
LOG_DIR = "bayes_test_rep_gen_46_4_100_epoch_"
max_trials_val = 100
# int(max_trials_val/10)
tuner_bo = BayesianOptimization(
            hypermodel,
            objective='val_mae',
            max_trials=max_trials_val,
            seed=1,
            executions_per_trial=2,
            num_initial_points = int(max_trials_val/10),
            directory = LOG_DIR)
tuner_bo.search(x=X_train, y=Y_train,
             epochs=100,
             batch_size=2000,
             validation_data=(X_test, Y_test))

In [ ]:
tuner.results_summary()

In [ ]:
best_model = tuner.get_best_models(num_models=1)

In [ ]:
best_model[0].summary()

In [ ]:
best_model[0].save('best_model_2_1_3ed_fns_5_mat')

In [ ]:
loaded_model = keras.models.load_model("best_model_2_1_3ed_fns_5_mat")

In [ ]:
loaded_model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=1000, batch_size=2500, verbose=1)

In [ ]:
test_pred = loaded_model.predict(X_test)

In [ ]:
outputfile = open("out_2.csv", "w")
for _ in test_pred:
    str_ =str(_[0])+"\n"
    outputfile.write(str_)
outputfile.close()